In [58]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
from config import mongoConn

In [59]:
# URL of page to be scraped
query = 'Amelie'
base_url = 'https://reelgood.com'
url = f'{base_url}/search?q={query}'
print(url)

'https://reelgood.com/search?q=Amelie'

In [70]:
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
link = soup.select('.e1qyeclq5 a')[0]['href']
show_url = base_url + link
print(show_url)

https://reelgood.com/movie/amelie-2001


In [71]:
show_response = requests.get(show_url)
show_soup = BeautifulSoup(show_response.text, 'html.parser') 

In [72]:
#services = show_soup.select('.e126mwsw1 span[class*=hou11]')
services = show_soup.select('.e126mwsw1 span[class*=hou113]') 

In [64]:
title = ''
desc = '' 
feature_img = '' 
meta = ''
recommended = ''
try:
    title = show_soup.select('h1.e14injhv7')[0].text
    desc = show_soup.select('p[itemprop=description]')[0].text 
    feature_img = show_soup.select('.e1x40mdt0 picture.e1181ybh0 img.e1181ybh1')[0]['src']  
    meta = show_soup.select('.ey4ir3j2') 
    recommended = show_soup.select('.e1yfir8f4 .e1qyeclq4') 
except:
    print('error') 

In [65]:
dic_list = {
    'query': query,
    'services': [s.text for s in set(services)],
    'title': title,
    'description': desc,
    'feature_img': feature_img ,
    'info': '',
    'recommended':[r.text for r in recommended],
    'meta': ''
} 
#dic_list

In [66]:
#conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(mongoConn) 
db = client.shows_db
collection = db.items
collection.insert_one(dic_list)

In [68]:
results = collection.find({}, {'_id': False})
coll_df = pd.DataFrame(results)
coll_json = coll_df.to_json(orient='records') 
coll_json

'[{"query":"cobra kai","services":["Netflix","Rent or Buy","YouTube Premium"],"title":"Cobra Kai","description":"This Karate Kid sequel series picks up 30 years after the events of the 1984 All Valley Karate Tournament and finds Johnny Lawrence on the hunt for redemption by reopening the infamous Cobra Kai karate dojo. This reignites his old rivalry with the successful Daniel LaRusso, who has been working to maintain the balance in his life without mentor Mr. Miyagi.Cobra Kai featuring Ralph Macchio and William Zabka has one or more episodes streaming with subscription on Netflix, streaming with subscription on YouTube Premium, free on YouTube Premium, and 5 others. It\'s an action & adventure and comedy show with 30 episodes over 3 seasons. Cobra Kai is still airing with no announced date for the next episode or season. It has a high IMDb audience rating of 8.6 (103,319 votes) and was very well received by critics.","feature_img":"https:\\/\\/img.reelgood.com\\/content\\/show\\/d58a63